<a href="https://colab.research.google.com/github/joselopez11c/mi-primer-proyecto/blob/main/remates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber
!pip install openpyxl

import requests
import re
import os
import pandas as pd
import pdfplumber
from bs4 import BeautifulSoup
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 43.3 MB/s eta 0:00:00


In [ ]:
url = 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml'
urlLogin = 'https://remaju.pj.gob.pe/remaju/pages/seguridad/login.xhtml'

headers = {
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9,es;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Faces-Request': 'partial/ajax',
    'Origin': 'https://remaju.pj.gob.pe',
    'Referer': 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

response = requests.get(urlLogin, headers=headers)
cookie_header = 'JSESSIONID=' + response.cookies.get_dict().get('JSESSIONID')
headers['Cookie'] = cookie_header

def fetch_page(first_row):
  dataPagination = {
      'javax.faces.partial.ajax': 'true',
      'javax.faces.source': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.execute': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.render': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.behavior.event': 'page',
      'javax.faces.partial.event': 'page',
      'formBuscarRemateExterno:listaRemate_pagination': 'true',
      'formBuscarRemateExterno:listaRemate_skipChildren': 'true',
      'formBuscarRemateExterno:listaRemate_first': str(first_row),
      'formBuscarRemateExterno:listaRemate_rows': '12',
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
  }
  response = requests.post(url, headers=headers, data=dataPagination)
  return response

def downloadFile(idFile):
  headers = {
      'Accept': 'application/xml, text/xml, */*; q=0.01',
      'Content-Type': 'application/x-www-form-urlencoded',
      'Cookie': cookie_header
  }

  dataFile = {
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
      idFile: '',
      'javax.faces.ViewState': view_state_value
  }

  response = requests.post(url, headers=headers, data=dataFile)
  return response


page = fetch_page(24) ####################################################### LLAMADA A LA PAGINA

def parse_page_content(page_content):
    soup = BeautifulSoup(page_content, 'xml')
    view_state_value = soup.find('update', {'id': 'j_id1:javax.faces.ViewState:0'}).text
    cdata_content = soup.find('update').text
    return BeautifulSoup(cdata_content, 'html.parser'), view_state_value

# Ahora parseamos el contenido del CDATA, que es HTML
soup_html, view_state_value = parse_page_content(page.content)


# Buscar todos los div con la clase 'card azul'
def extract_card_data(soup_html, view_state_value, cookie_header):
  cards = soup_html.find_all('div', class_='card azul')
  data = []
  for card in cards:
      # Obtener número de remate y convocatoria
      remate_info = card.find('span', class_='text-bold label-danger h6').get_text(strip=True) if card.find('span', class_='text-bold label-danger h6') else 'No disponible'

      # Dividir la información de remate en número y convocatoria
      if remate_info != 'No disponible':
          parts = remate_info.split(" - ")
          remate_number = parts[0] if len(parts) > 0 else 'No disponible'
          convocatoria = parts[1] if len(parts) > 1 else 'No disponible'
      else:
          remate_number = 'No disponible'
          convocatoria = 'No disponible'

      # Ubicación, buscando la etiqueta específica y extrayendo el texto siguiente
      location_tag = card.find('i', class_='fa fa-map-marker')
      location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'

      # Descripción del inmueble, buscando el contenedor específico
      description = card.find('div', class_='texto-info-scroll').get_text(strip=True) if card.find('div', class_='texto-info-scroll') else 'No disponible'

      # Buscar todos los scripts dentro del card
      scripts = card.find_all('script')
      script_id = scripts[-1].get('id')[:-2] if scripts and scripts[-1].get('id').endswith('_s') else 'Script ID no encontrado'

      # Guardar los datos extraídos en la lista
      data.append({
          'Numer': remate_number,
          'Convocation': convocatoria,
          'Location': location,
          'Description': description,
          'Script ID': script_id,
          'View Code' : view_state_value,
          'Cookie header' : cookie_header
      })
  return data

data = extract_card_data(soup_html, view_state_value, headers['Cookie'])

# Crear un DataFrame con los datos recolectados
df = pd.DataFrame(data)

df['Demandante'] = ""
df['Demandado'] = ""
df['Expediente_Judicial'] = ""
df['Juzgado'] = ""

drive.mount('/content/drive')
save_folder = '/content/drive/My Drive/remates'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)


for index, row in df.iterrows():
    file_name = row['Numer'] + '.pdf'
    response = downloadFile(row['Script ID'])

    if response.ok and 'application/pdf' in response.headers.get('Content-Type', ''):
        pdf_path = os.path.join(save_folder, file_name)
        with open(pdf_path, 'wb') as f:
            f.write(response.content)

        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
              text += page.extract_text().replace('\n', ' ') if page.extract_text() else ''

        # Usar expresiones regulares para extraer la información
        demandante = re.search(r"seguidos por (.*?),", text, re.DOTALL)
        demandado = re.search(r", contra (.*?), sobre", text, re.DOTALL)
        expediente = re.search(r"Expediente Judicial (.*?),", text, re.DOTALL)
        juzgado = re.search(r"tramitado ante el (.*?),", text, re.DOTALL)

        df.at[index, 'Demandante'] = demandante.group(1) if demandante else "No encontrado"
        df.at[index, 'Demandado'] = demandado.group(1) if demandado else "No encontrado"
        df.at[index, 'Expediente_Judicial'] = expediente.group(1) if expediente else "No encontrado"
        df.at[index, 'Juzgado'] = juzgado.group(1).replace('\n', ' ').strip() if juzgado else "No encontrado"

        if "BANCO DE CREDITO DEL PERU" not in demandante.group(1) if demandante else "":
            os.remove(pdf_path)

    else:
        print("Failed to download a valid PDF file.")

df = filtered_df = df[df['Demandante'].str.contains("BANCO DE CREDITO DEL PERU", na=False)]

columns_to_include = ['Numer', 'Demandante', 'Demandado', 'Expediente_Judicial', 'Juzgado']
df_to_save = df[columns_to_include]

excel_path = os.path.join(save_folder, 'Remates_Data.xlsx')

# Usando openpyxl como motor para escribir archivos .xlsx
df_to_save.to_excel(excel_path, index=False, engine='openpyxl')

df_to_save

In [24]:
url = 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml'
urlLogin = 'https://remaju.pj.gob.pe/remaju/pages/seguridad/login.xhtml'

def initialize_session(url_login):
  headers = {
      'Accept': 'application/xml, text/xml, */*; q=0.01',
      'Accept-Language': 'en-US,en;q=0.9,es;q=0.8',
      'Connection': 'keep-alive',
      'Content-Type': 'application/x-www-form-urlencoded',
      'DNT': '1',
      'Faces-Request': 'partial/ajax',
      'Origin': 'https://remaju.pj.gob.pe',
      'Referer': 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml',
      'Sec-Fetch-Dest': 'empty',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Site': 'same-origin',
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
      'X-Requested-With': 'XMLHttpRequest',
  }
  response = requests.get(url_login, headers=headers)
  headers['Cookie'] = 'JSESSIONID=' + response.cookies.get_dict().get('JSESSIONID')
  return headers

def fetch_page(headers, first_row):
  dataPagination = {
      'javax.faces.partial.ajax': 'true',
      'javax.faces.source': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.execute': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.render': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.behavior.event': 'page',
      'javax.faces.partial.event': 'page',
      'formBuscarRemateExterno:listaRemate_pagination': 'true',
      'formBuscarRemateExterno:listaRemate_skipChildren': 'true',
      'formBuscarRemateExterno:listaRemate_first': str(first_row),
      'formBuscarRemateExterno:listaRemate_rows': '12',
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
  }
  return requests.post(url, headers=headers, data=dataPagination)

def downloadFile(cookie_header, idFile, view_state_value):
  headers = {
      'Accept': 'application/xml, text/xml, */*; q=0.01',
      'Content-Type': 'application/x-www-form-urlencoded',
      'Cookie': cookie_header
  }

  dataFile = {
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
      idFile: '',
      'javax.faces.ViewState': view_state_value
  }

  return requests.post(url, headers=headers, data=dataFile)


def parse_page_content(page_content):
  soup = BeautifulSoup(page_content, 'xml')
  view_state_value = soup.find('update', {'id': 'j_id1:javax.faces.ViewState:0'}).text
  cdata_content = soup.find('update').text
  return BeautifulSoup(cdata_content, 'html.parser'), view_state_value

def extract_pdf_data(pdf_path):
  text = ""
  with pdfplumber.open(pdf_path) as pdf:
      for page in pdf.pages:
          text += page.extract_text().replace('\n', ' ') if page.extract_text() else ''
  return text

# Buscar todos los div con la clase 'card azul'
def extract_card_data(soup_html, view_state_value, cookie_header):
  cards = soup_html.find_all('div', class_='card azul')
  data = []
  for card in cards:
    # Obtener número de remate y convocatoria
    remate_info = card.find('span', class_='text-bold label-danger h6').get_text(strip=True) if card.find('span', class_='text-bold label-danger h6') else 'No disponible'

    # Usar expresiones regulares para extraer solo el número de remate
    match = re.search(r'Remate N° (\d+)', remate_info)
    if match:
        remate_number = f"N° {match.group(1)}"  # Esto guardará "N° 14066"
    else:
        remate_number = 'No disponible'

    # Dividir la información de remate en número y convocatoria, si es necesario
    parts = remate_info.split(" - ") if remate_info != 'No disponible' and len(remate_info.split(" - ")) > 1 else ['No disponible', 'No disponible']
    convocatoria = parts[1] if len(parts) > 1 else 'No disponible'

    # Ubicación, buscando la etiqueta específica y extrayendo el texto siguiente
    location_tag = card.find('i', class_='fa fa-map-marker')
    location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'

    # Descripción del inmueble, buscando el contenedor específico
    description = card.find('div', class_='texto-info-scroll').get_text(strip=True) if card.find('div', class_='texto-info-scroll') else 'No disponible'

    # Buscar todos los scripts dentro del card
    scripts = card.find_all('script')
    script_id = scripts[-1].get('id')[:-2] if scripts and scripts[-1].get('id').endswith('_s') else 'Script ID no encontrado'

    # Guardar los datos extraídos en la lista
    data.append({
        'Number': remate_number,
        'Convocation': convocatoria,
        'Location': location,
        'Description': description,
        'Script ID': script_id,
        'View Code' : view_state_value,
        'Cookie header' : cookie_header
    })
  return data

def update_df_with_pdf_data(df, index, pdf_text):
  # Utilizar expresiones regulares para extraer la información del PDF
  demandante_regex = re.search(r"seguidos por (.*?), contra", pdf_text, re.DOTALL)
  demandado_regex = re.search(r", contra (.*?), sobre", pdf_text, re.DOTALL)
  expediente_regex = re.search(r"Expediente Judicial (.*?), tramitado", pdf_text, re.DOTALL)
  juzgado_regex = re.search(r"tramitado ante el (.*?), a cargo", pdf_text, re.DOTALL)

  # Actualizar el DataFrame con los datos extraídos
  df.at[index, 'Demandante'] = demandante_regex.group(1) if demandante_regex else "No encontrado"
  df.at[index, 'Demandado'] = demandado_regex.group(1) if demandado_regex else "No encontrado"
  df.at[index, 'Expediente_Judicial'] = expediente_regex.group(1) if expediente_regex else "No encontrado"
  df.at[index, 'Juzgado'] = juzgado_regex.group(1).replace('\n', ' ').strip() if juzgado_regex else "No encontrado"

  # Adicionalmente, se puede realizar una acción específica si un criterio particular no se cumple
  # Por ejemplo, eliminar el archivo PDF si el demandante no es el esperado
  if "BANCO DE CREDITO DEL PERU" not in (demandante_regex.group(1) if demandante_regex else ""):
      os.remove(os.path.join('/content/drive/My Drive/remates', f"{df.at[index, 'Number']}.pdf"))

def process_pdf_and_update_df(headers, df, save_folder):
  for index, row in df.iterrows():
      file_name = f"{row['Number']}.pdf"
      file_response = downloadFile(headers['Cookie'], row['Script ID'], row['View Code'])
      if file_response.ok and 'application/pdf' in file_response.headers.get('Content-Type', ''):
          pdf_path = os.path.join(save_folder, file_name)
          with open(pdf_path, 'wb') as f:
              f.write(file_response.content)
          pdf_text = extract_pdf_data(pdf_path)
          update_df_with_pdf_data(df, index, pdf_text)
      else:
          print("Failed to download a valid PDF file.")
  return df


def process_and_save_data(df, save_folder):
  #print('Demandante: ' + df['Demandante'])
  df_filtered = df[df['Demandante'].str.contains("BANCO DE CREDITO DEL PERU", na=False)]
  columns_to_include = ['Number', 'Demandante', 'Demandado', 'Expediente_Judicial', 'Juzgado']
  df_to_save = df_filtered[columns_to_include]
  excel_path = os.path.join(save_folder, 'Remates_Data.xlsx')
  df_to_save.to_excel(excel_path, index=False, engine='openpyxl')
  return df_to_save


def check_for_stop_condition(df, stop_number):
  return any(df['Number'] == stop_number)


stop_number = "N° 14039"  # Aquí defines el número en el que quieres detener la búsqueda

headers = initialize_session(urlLogin)
save_folder = '/content/drive/My Drive/remates'
drive.mount('/content/drive')
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

df = pd.DataFrame()
stop_found = False
first_row = 0

while not stop_found:
    print(f"Fetching page starting at row {first_row}")
    page_response = fetch_page(headers, first_row)
    soup_html, view_state_value = parse_page_content(page_response.content)
    page_data = extract_card_data(soup_html, view_state_value, headers['Cookie'])
    page_df = pd.DataFrame(page_data)
    df = pd.concat([df, page_df], ignore_index=True)

    # Check if the stop condition is met
    if check_for_stop_condition(page_df, stop_number):
        print(f"Stop number {stop_number} found. Stopping the search.")
        stop_found = True

    # Preparar para la siguiente página
    first_row += 12

# Continuar con el procesamiento de PDF y guardado de datos solo si se necesita
if stop_found:
    df = process_pdf_and_update_df(headers, df, save_folder)
    df_to_save = process_and_save_data(df, save_folder)
    df_to_save
else:
    print("The specified stop number was not found in the pages fetched.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fetching page starting at row 0


<ipython-input-24-4af213e60fd3>:90: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'


Fetching page starting at row 12
Fetching page starting at row 24
Stop number N° 14039 found. Stopping the search.
